In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('Dataset/train.csv')
test = pd.read_csv('Dataset/test.csv')

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [15]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
train.corr()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
toxic,1.000000,0.308619,0.676515,0.157058,0.647518,0.266009
severe_toxic,0.308619,1.000000,0.403014,0.123601,0.375807,0.201600
obscene,0.676515,0.403014,1.000000,0.141179,0.741272,0.286867
threat,0.157058,0.123601,0.141179,1.000000,0.150022,0.115128
insult,0.647518,0.375807,0.741272,0.150022,1.000000,0.337736
identity_hate,0.266009,0.201600,0.286867,0.115128,0.337736,1.000000


In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [12]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [20]:
train_comments = train['comment_text'].values
test_comments = test['comment_text'].values
train_comments_lengths = [len(c) for c in train_comments]
test_comments_lengths = [len(c) for c in test_comments]

In [21]:
def explore_comments(arr):
    print("Max-length:", np.max(arr))
    print("Avg-length:", np.average(arr))
    print("Min-length:", np.min(arr))
    print("Standard division", np.std(arr))
    print("Range", np.min(arr), np.average(arr) + 2 * np.std(arr))

print("---Train---")
explore_comments(train_comments_lengths)

print("---Test---")
explore_comments(test_comments_lengths) # select 1600 for Char RNN I guess

---Train---
Max-length: 5000
Avg-length: 394.0735221312143
Min-length: 6
Standard division 590.7191118675463
Range 6 1575.511745866307
---Test---
Max-length: 5145
Avg-length: 364.87636781489124
Min-length: 1
Standard division 592.4999179054861
Range 1 1549.8762036258634


In [5]:
df = pd.concat([train['comment_text'], test['comment_text']], axis=0)
df = df.fillna("unknown") # one na
nrow_train = train.shape[0]

In [6]:
print(df.shape[0])

312735


In [7]:
df.head()

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: comment_text, dtype: object

In [14]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=50000)
data = vectorizer.fit_transform(df)

In [18]:
print(data)

  (0, 30948)	0.303969287271
  (0, 25184)	0.439310327854
  (0, 19033)	0.472371430201
  (0, 38801)	0.23236342233
  (0, 45516)	0.256178219924
  (0, 26627)	0.226316440917
  (0, 2149)	0.250775225367
  (0, 44150)	0.507687851617
  (1, 46862)	0.46856736625
  (1, 32514)	0.356350091178
  (1, 13443)	0.330073205688
  (1, 15110)	0.293417682828
  (1, 10939)	0.425337538919
  (1, 6677)	0.398107959631
  (1, 15119)	0.344985519345
  (2, 34267)	0.357961562327
  (2, 37287)	0.128627785498
  (2, 39551)	0.127969949
  (2, 2353)	0.122357377887
  (2, 25134)	0.166687387398
  (2, 41511)	0.315480652312
  (2, 25280)	0.110393552134
  (2, 35120)	0.150295505425
  (2, 13450)	0.141968469372
  (2, 28210)	0.150592886875
  :	:
  (322848, 48108)	0.273934001447
  (322848, 25941)	0.106777082139
  (322848, 10992)	0.106751227899
  (322848, 1393)	0.167945309328
  (322848, 27407)	0.241578176495
  (322848, 782)	0.121462592248
  (322848, 37417)	0.114048482213
  (322848, 16989)	0.117439822374
  (322848, 37423)	0.128456747746
  (32284

In [28]:
X = MaxAbsScaler().fit_transform(data)
col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((test.shape[0], len(col)))

In [30]:
print(X)

  (0, 30948)	0.303969287271
  (0, 25184)	0.541560182873
  (0, 19033)	0.53842074936
  (0, 38801)	0.23236342233
  (0, 45516)	0.256178219924
  (0, 26627)	0.226316440917
  (0, 2149)	0.250775225367
  (0, 44150)	0.507687851617
  (1, 46862)	0.46856736625
  (1, 32514)	0.356350091178
  (1, 13443)	0.330073205688
  (1, 15110)	0.293417682828
  (1, 10939)	0.612548682372
  (1, 6677)	0.398107959631
  (1, 15119)	0.344985519345
  (2, 34267)	0.357961562327
  (2, 37287)	0.128627785498
  (2, 39551)	0.127969949
  (2, 2353)	0.122357377887
  (2, 25134)	0.18505098662
  (2, 41511)	0.365669200524
  (2, 25280)	0.110393552134
  (2, 35120)	0.150295505425
  (2, 13450)	0.141968469372
  (2, 28210)	0.150592886875
  :	:
  (322848, 48108)	0.273934001447
  (322848, 25941)	0.106777082139
  (322848, 10992)	0.194399283012
  (322848, 1393)	0.21447014178
  (322848, 27407)	0.241578176495
  (322848, 782)	0.121462592248
  (322848, 37417)	0.114048482213
  (322848, 16989)	0.117439822374
  (322848, 37423)	0.156609739444
  (322848, 